In [ ]:
#Standard inmorts 
import numpy as np
import pandas as pd
import os
import feather
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import pysubgroup as ps
import graphviz
from scipy import io
from scipy import stats

import warnings
from pandas.core.common import SettingWithCopyWarning



#the rest should be ML 

from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# K-Means 
from sklearn.cluster import KMeans


# Pipeline stuff
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn import tree
import pydotplus

## settings

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=FutureWarning)

# 1. Read Data

In [ ]:
df1 = pd.read_feather('../Data3/D3_EK_CL.ftr', columns=None, use_threads=True)

### Data inspection

In [ ]:
df1.head(5)

In [ ]:
print(df1.shape)

In [ ]:
df1['kproto_labels'] = df1['kproto_labels'].astype('category')
df1['dbscan_gower'] = df1['dbscan_gower'].astype('category')
df1['AgloSngl_gower'] = df1['AgloSngl_gower'].astype('category')
df1['AgloAvg_gower'] = df1['AgloAvg_gower'].astype('category')
df1['AgloComp_gower'] = df1['AgloComp_gower'].astype('category')

In [ ]:
df1.dtypes

# Filtering

In [ ]:
df2 = df1.drop(['EBELN','EBELP','TXZ01','EMATN','INFNR','BPRME','BEDAT'], axis=1).copy()

In [ ]:
df2.head(4)

# precompute Dataframe

In [ ]:
df_kproto = pd.concat([df2.iloc[: , :-6], df2.loc[:, ['kproto_labels']]],axis=1)

In [ ]:
# label festlegen
df_kproto['target'] = df_kproto['kproto_labels'].copy()
df_kproto = df_kproto.drop('kproto_labels', axis= 1).copy()

In [ ]:
df_kproto.head(2)

In [ ]:
df_dbscan = pd.concat([df2.iloc[: , :-5], df2.loc[:, ['dbscan_gower']]],axis=1)
# label festlegen
df_dbscan['target'] = df_dbscan['dbscan_gower'].copy()
df_dbscan = df_dbscan.drop('dbscan_gower', axis= 1).copy()

In [ ]:
df_dbscan.head(2)

In [ ]:
df_aglo = pd.concat([df2.iloc[: , :-5], df2.loc[:, ['AgloComp_gower']]],axis=1)
# label festlegen
df_aglo['target'] = df_aglo['AgloComp_gower'].copy()
df_aglo = df_aglo.drop('AgloComp_gower', axis= 1).copy()

In [ ]:
df_aglo.head(2)

# Explaining Clusters with Decision Trees

For a qualitative evaluation of the clusters' key characteristcs, Decision Trees can be used to inspect each cluster individually. A Decision Tree is traided using the cluster label as target variable. Subsequently, the decision tree can be inspected visually.

In [ ]:
def data_prep_explanation(df):
    df = df.copy()
    num_cluster = df.target.unique()
    
    #select categorical and numerical columns
    cats_cat = df.select_dtypes(include=['category']).columns
    nums_cat = df.select_dtypes(include=['float64', 'int64']).columns
    
    # encode target variable column 
    t = make_column_transformer(
    (OneHotEncoder(), ['target']), remainder='passthrough')

    transformed = t.fit_transform(df)
    transformed_df = pd.DataFrame(transformed, columns=t.get_feature_names())
    
    if len(num_cluster) == 3:
        cluster_0 = transformed_df.drop(transformed_df.columns[[1,2]], axis=1).copy()
        cluster_1 = transformed_df.drop(transformed_df.columns[[0,2]], axis=1).copy()
        cluster_2 = transformed_df.drop(transformed_df.columns[[0,1]], axis=1).copy()
        cluster_3 = cluster_2.copy()
    
    elif len(num_cluster) == 4:
        cluster_0 = transformed_df.drop(transformed_df.columns[[1,2,3]], axis=1).copy()
        cluster_1 = transformed_df.drop(transformed_df.columns[[0,2,3]], axis=1).copy()
        cluster_2 = transformed_df.drop(transformed_df.columns[[0,1,3]], axis=1).copy()
        cluster_3 = transformed_df.drop(transformed_df.columns[[0,1,2]], axis=1).copy()
        
    else:
        None
    
    return cluster_0, cluster_1, cluster_2, cluster_3

def prep_for_dt(df):
    clusters = data_prep_explanation(df)
    t_X = []
    t_y = []
    for i in range(len(clusters)):
        cluster = clusters[i]
        X = cluster.iloc[:,1:]
        X['MENGE'] = X['MENGE'].astype('int64').copy()
        X['NETPR'] = X['NETPR'].astype('float64').copy()
        X['NETWR'] = X['NETWR'].astype('float64').copy()
        y = cluster.iloc[:,0]

        # encode categorical features of X
        cat = X.select_dtypes(include=['object']).columns.tolist()

        transformer = make_column_transformer(
            (OrdinalEncoder(), cat),
            remainder='passthrough')

        transformed = transformer.fit_transform(X)
        transformed_X = pd.DataFrame(transformed, columns=X.columns)
        transformed_X['MENGE'] = X['MENGE']
        transformed_X['NETPR'] = X['NETPR']
        transformed_X['NETWR'] = X['NETWR']

        # encode target feature
        label_encoder = LabelEncoder()
        label_encoder.fit(y)
        transformed_y = label_encoder.transform(y)

        t_X.append(transformed_X)
        t_y.append(transformed_y)
        
    return t_X, t_y

def decision_tree_explanation(df):
    graphs = []

    prepared_data = prep_for_dt(df)
    for i in range(len(prepared_data[0])):
        X = prepared_data[0][i]
        y = prepared_data[1][i]
        
        # DecisionTreeClassifier
        dt = tree.DecisionTreeClassifier(random_state=42)
        dt = dt.fit(X,y)
        dtg = tree.export_graphviz(dt, 
                                   out_file=None, 
                                   feature_names=X.columns, 
                                   #filled=True, 
                                   class_names=True, 
                                   rounded=True)
        graph = graphviz.Source(dtg)
        graphs.append(graph)
        
    return graphs

def colored_tree_explanation(df):
    graphs = []

    prepared_data = prep_for_dt(df)
    for i in range(len(prepared_data[0])):
        X = prepared_data[0][i]
        y = prepared_data[1][i]
        
        # DecisionTreeClassifier
        dt = tree.DecisionTreeClassifier(random_state=42)
        dt = dt.fit(X,y)
        
        colors = ['#FEAEFE', '#FC05FB', '#B4FFE4', '#3FFEBA', '#1F363D', '#2B4570', '3A2D32', '706F6F']
        
        dtg = tree.export_graphviz(dt, 
                                   out_file=None, 
                                   feature_names=X.columns, 
                                   filled=True, 
                                   class_names=True,
                                   rounded=True)
        graph = pydotplus.graph_from_dot_data(dtg)
        nodes = graph.get_node_list()
        for node in nodes: 
            node.set_fillcolor(colors[5])
        # change color of nodes according to class 
#        for node in nodes:
#            if node.get_name() not in ('node', 'edge', '"\\n"'):
#                values = clf.tree_.value[int(node.get_name())][0]
                
#                #color only nodes where only one class is present
#                if max(values) == sum(values):    
#                    node.set_fillcolor(colors[np.argmax(values)])
                    
                #mixed nodes get the default color
#                else:
#                    node.set_fillcolor(colors[-1])
        graphs.append(graph)
        
    return graphs

In [ ]:
X = prepared_data[0][i]
y = prepared_data[1][i]

# DecisionTreeClassifier
dt = tree.DecisionTreeClassifier(random_state=42)
dt = dt.fit(X,y)
dtg = tree.export_graphviz(dt, 
                           out_file=None, 
                           feature_names=X.columns, 
                           filled=True, 
                           class_names=True, 
                           rounded=True)
graph = graphviz.Source(dtg)


### Agglomerative Cluster Inspection

In [ ]:
df_aglo.target.unique()

In [ ]:
num_cluster = len(df_aglo.target.unique())
print(num_cluster)

In [ ]:
graphs = decision_tree_explanation(df_aglo)

In [ ]:
graphs[0]

In [ ]:
graphs[1]

In [ ]:
graphs[2]

#### Color Scheme - Cluster Inspection Agglomerative Clustering

In [ ]:
graphs = colored_tree_explanation(df_aglo)
counter = 0
for graph in graphs:
    graph_name = "col_cluster_aglo" + str(counter) + ".svg"
    path = "/home/weclapp/PM Preprocessing Paper/with Gower/exports/"
    graph.write_svg(path + graph_name)
    counter = counter+1

### DBSCAN Cluster Inspection

In [ ]:
df_dbscan.target.unique()

#### Color Scheme - Cluster Inspection DBSCAN Clustering

In [ ]:
#graphs = colored_tree_explanation(df_dbscan)
#counter = 0
#for graph in graphs:
#    graph_name = "col_cluster_dbscan" + str(counter) + ".svg"
#    path = "/home/weclapp/PM Preprocessing Paper/with Gower/exports/"
#    graph.write_svg(path + graph_name)
#    counter = counter+1

### KPrototype Cluster Inspection

In [ ]:
df_kproto.target.unique()

In [ ]:
graphs = decision_tree_explanation(df_kproto)

In [ ]:
graphs[0]

In [ ]:
graphs[1]

In [ ]:
graphs[2]

***

***

# Manual Data Preparation

In [ ]:
df = df_kproto.copy()

In [ ]:
df.target.unique()

In [ ]:
cats_cat = df.select_dtypes(include=['category']).columns
nums_cat = df.select_dtypes(include=['float64', 'int64']).columns

In [ ]:
# encode target variable 
t = make_column_transformer(
    (OneHotEncoder(), ['target']), remainder='passthrough')

transformed = t.fit_transform(df)
transformed_df = pd.DataFrame(transformed, columns=t.get_feature_names())
print(transformed_df.head())

In [ ]:
# split df in three datasets 
cluster_0 = transformed_df.drop(['onehotencoder__x0_1', 'onehotencoder__x0_2'], axis=1).copy()
cluster_1 = transformed_df.drop(['onehotencoder__x0_0', 'onehotencoder__x0_2'], axis=1).copy()
cluster_2 = transformed_df.drop(['onehotencoder__x0_1', 'onehotencoder__x0_0'], axis=1).copy()

cluster_0.head(2)

In [ ]:
cluster_0 = transformed_df.drop(transformed_df.columns[[1,2]], axis=1).copy()
cluster_0.head(2)

### Explanation of Agglomerative Cluster 0 with Decision Tree Classifier

In [ ]:
# prepare data for DT

X = cluster_0.iloc[:,1:]
y = cluster_0.iloc[:,0]

In [ ]:
X['MENGE'] = X['MENGE'].astype('int64')
X['NETPR'] = X['NETPR'].astype('float64')
X['NETWR'] = X['NETWR'].astype('float64')

In [ ]:
# encode categorical features of X
cat = X.select_dtypes(include=['object']).columns.tolist()


transformer = make_column_transformer(
    (OrdinalEncoder(), cat),
    remainder='passthrough')

transformed = transformer.fit_transform(X)
transformed_X = pd.DataFrame(transformed, columns=X.columns)
transformed_X['MENGE'] = X['MENGE']
transformed_X['NETPR'] = X['NETPR']
transformed_X['NETWR'] = X['NETWR']

# encode target feature
label_encoder = LabelEncoder()
label_encoder.fit(y)
transformed_y = label_encoder.transform(y)

In [ ]:
dt = tree.DecisionTreeClassifier()
dt = dt.fit(transformed_X,transformed_y)

In [ ]:
dtg = tree.export_graphviz(dt, out_file=None, feature_names=X.columns, filled=True, class_names=True)
graph = graphviz.Source(dtg)
graph

### Explanation of Agglomerative Cluster 1 with Decision Tree Classifier

In [ ]:
# prepare data for DT

X = cluster_1.iloc[:,1:]
y = cluster_1.iloc[:,0]

In [ ]:
X['MENGE'] = X['MENGE'].astype('int64').copy()
X['NETPR'] = X['NETPR'].astype('float64').copy()
X['NETWR'] = X['NETWR'].astype('float64').copy()

In [ ]:
# encode categorical features of X
cat = X.select_dtypes(include=['object']).columns.tolist()


transformer = make_column_transformer(
    (OrdinalEncoder(), cat),
    remainder='passthrough')

transformed = transformer.fit_transform(X)
transformed_X = pd.DataFrame(transformed, columns=X.columns)
transformed_X['MENGE'] = X['MENGE']
transformed_X['NETPR'] = X['NETPR']
transformed_X['NETWR'] = X['NETWR']

# encode target feature
label_encoder = LabelEncoder()
label_encoder.fit(y)
transformed_y = label_encoder.transform(y)

In [ ]:
dt = tree.DecisionTreeClassifier()
dt = dt.fit(transformed_X,transformed_y)

In [ ]:
dtg = tree.export_graphviz(dt, out_file=None, feature_names=X.columns, filled=True, class_names=True)
graph = graphviz.Source(dtg)
graph

### Explanation of Agglomerative Cluster 2 with Decision Tree Classifier

In [ ]:
# prepare data for DT

X = cluster_2.iloc[:,1:]
y = cluster_2.iloc[:,0]

In [ ]:
X['MENGE'] = X['MENGE'].astype('int64')
X['NETPR'] = X['NETPR'].astype('float64')
X['NETWR'] = X['NETWR'].astype('float64')

In [ ]:
# encode categorical features of X
cat = X.select_dtypes(include=['object']).columns.tolist()


transformer = make_column_transformer(
    (OrdinalEncoder(), cat),
    remainder='passthrough')

transformed = transformer.fit_transform(X)
transformed_X = pd.DataFrame(transformed, columns=X.columns)
transformed_X['MENGE'] = X['MENGE']
transformed_X['NETPR'] = X['NETPR']
transformed_X['NETWR'] = X['NETWR']

# encode target feature
label_encoder = LabelEncoder()
label_encoder.fit(y)
transformed_y = label_encoder.transform(y)

In [ ]:
dt = tree.DecisionTreeClassifier()
dt = dt.fit(transformed_X,transformed_y)

In [ ]:
dtg = tree.export_graphviz(dt, out_file=None, feature_names=X.columns, filled=True, class_names=True)
graph = graphviz.Source(dtg)
graph